1.Submit a Google Colab notebook containing your completed code and experimentation results.

2.Include comments and explanations in your code to help understand the implemented logic.

**Additional Notes:**
*   Ensure that the notebook runs successfully in Google Colab.
*   Document any issues encountered during experimentation and how you addressed them.

**Grading:**
*   Each task will be graded out of the specified points.
*   Points will be awarded for correctness, clarity of code, thorough experimentation, and insightful analysis.

In [9]:
from google.colab import drive
# drive.mount('/content/drive')

# SOURCE_DIR = "/content/drive/MyDrive/Uni/NLP/HW2_NLP_4022/Q3_data.csv"
SOURCE_DIR = 'Q3_data.csv'

In [10]:
import torch
import re
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import math
from gensim.models import Word2Vec

In [11]:
def delete_hashtag_usernames(text):
  try:
    result = []
    for word in text.split():
      if word[0] not in ['@', '#']:
        result.append(word)
    return ' '.join(result)
  except:
    return ''

def delete_url(text):
  text = re.sub(r'http\S+', '', text)
  return text

def delete_ex(text):
  text = re.sub(r'\u200c', '', text)
  return text

# 0. Data preprocessing

In [12]:
!pip install json-lines

In [13]:
import json_lines

In [15]:
# 1. extract all tweets from file and save them in memory
# 2. remove urls, hashtags and usernames. use the prepared functions

tweetsTable=pd.read_csv(SOURCE_DIR)
# rawTweets=tweetsTable['Text']
rawTweets=tweetsTable['PureText']
print(rawTweets[:5])
tweets=[]
for t in rawTweets:
  t=delete_hashtag_usernames(t)
  t=delete_url(t)
  t=delete_ex(t)
  tweets.append((t))
print(tweets[:3])

0             بنشین تا شود نقش فال ما نقش هم‌ فردا شدن
1    این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده...
2                               برای ایران، برای مهسا.
3                                      مرگ بر دیکتاتور
4                       نذاریم خونشون پایمال شه.‌‌.‌‌.
Name: PureText, dtype: object
['بنشین تا شود نقش فال ما نقش هم فردا شدن', 'این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده از بس پای منبر دستمال کشی کرده.', 'برای ایران، برای مهسا.']


# 1. Functions

## Cosine Similarity

To measure the similarity between two words, you need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows:

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

* $u \cdot v$ is the dot product (or inner product) of two vectors
* $||u||_2$ is the norm (or length) of the vector $u$
* $\theta$ is the angle between $u$ and $v$.
* The cosine similarity depends on the angle between $u$ and $v$.
    * If $u$ and $v$ are very similar, their cosine similarity will be close to 1.
    * If they are dissimilar, the cosine similarity will take a smaller value.

<img src="images/cosine_sim.png" style="width:800px;height:250px;">
<caption><center><font color='purple'><b>Figure 1</b>: The cosine of the angle between two vectors is a measure of their similarity.</font></center></caption>

Implement the function `cosine_similarity()` to evaluate the similarity between word vectors.

**Reminder**: The norm of $u$ is defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

In [24]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similarity between u and v

    Arguments:
        u -- a word vector of shape (n,)
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """

    dot_product = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u**2))
    norm_v = np.sqrt(np.sum(v**2))
    cosine_similarity = dot_product / (norm_u * norm_v)
    return cosine_similarity

def cosine_similarity_tensor(u, v):
  # same as previous function for tensors
    dot_product = torch.dot(u, v)
    norm_vec1 = torch.norm(u)
    norm_vec2 = torch.norm(v)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

## find k nearest neighbors

In [25]:
def find_k_nearest_neighbors(word, embedding_dict, k):
  """
    implement a function to return the nearest words to an specific word based on the given dictionary

    Arguments:
        word           -- a word, string
        embedding_dict -- dictionary that maps words to their corresponding vectors
        k              -- the number of word that should be returned

    Returns:
        a list of size k consisting of the k most similar words to the given word

    Note: use the cosine_similarity function that you have implemented to calculate the similarity between words
    """

  # check if word exists in the embedding dictionary
  if word not in embedding_dict:
        return []
  words_cosine_similarity = dict()
  # calculate similarity for all other words
  for token in embedding_dict.keys():
    words_cosine_similarity[token] = cosine_similarity(embedding_dict[word], embedding_dict[token]).item()
  # sort similarities and put them in a list
  words_cosine_similarity = dict(sorted(words_cosine_similarity.items(), key=lambda item: item[1]))
  return list(words_cosine_similarity.keys())[-k:][::-1]

def find_k_nearest_neighbors_tensor(word, embedding_dict, k):
  # same as previous function for tensors
  word_vector = embedding_dict[word]
  similarities = {}
  for key, vector in embedding_dict.items():
      if key != word:
          similarity = cosine_similarity_tensor(word_vector, vector)
          similarities[key] = similarity
  sorted_similarities_list = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
  neighbors = [item[0] for item in sorted_similarities_list[:k]]
  return neighbors


# 2. One hot encoding

In [ ]:
# 1. find one hot encoding of each word

# at first I used the onehotencoding of the sklearn:
# # tokenizing tweets into words
# tokenized_tweets=[]
# for tweet in tweets:
#   tokenized_tweets.append(tweet.split())
# # print(tokenized_tweets[5])

# # Flattening the list to words
# words = [word for sublist in tokenized_tweets for word in sublist]
# # print(words[0])

# # Reshape the words to be a column vector
# words_column_vector = np.array(words).reshape(-1, 1)
# # print(words_column_vector[:3])

# # Create the encoder
# encoder = OneHotEncoder(sparse_output=False)

# # Fit and transform the words to one-hot encoded vectors
# one_hot_encoded = encoder.fit_transform(words_column_vector)
# embedding_dict = {word: encoding for word, encoding in zip(words, one_hot_encoded)}
# k = 10
# word1 = "آزادی"
# nearest_words1 = find_k_nearest_neighbors(word1, embedding_dict, k)
# print(nearest_words1)

# first lets find the unique words
distinct_words = list(set([word for snt in tweets for word in snt.split(' ')]))
n = len(distinct_words)

#creating the embedding for each word and mark the used words as one. others as zero
one_hot = dict()
for i, word in enumerate(distinct_words):
  array = np.zeros(n)
  array[i] = 1
  one_hot.update({word : torch.tensor(array)})


In [ ]:
# 2. find 10 nearest words from "آزادی"

k = 10
word1 = "آزادی"
nearest_words1 = find_k_nearest_neighbors_tensor(word1, one_hot, k)
print(nearest_words1)

word2 = "مهسا"
nearest_words2 = find_k_nearest_neighbors_tensor(word2, one_hot, k)
print(nearest_words2)


['', 'ميچكد', 'اینفلوئنسرها', 'بندازن', 'موهایی', 'خوبه!؟', 'قشنگی😍🤝🏻', 'کیپاپرایم', 'ارشاد.', 'بازکرده.']
['', 'ميچكد', 'اینفلوئنسرها', 'بندازن', 'موهایی', 'خوبه!؟', 'قشنگی😍🤝🏻', 'کیپاپرایم', 'ارشاد.', 'بازکرده.']


In [ ]:
# seems like most values of the embedding is zero. lets check for none zero ones.
print(word1)
nonzero_index=0
for value in one_hot[word1]:
  if value != 0.0:
    print(nonzero_index,value)
  nonzero_index += 1

print(word2)
nonzero_index=0
for value in one_hot[word2]:
  if value != 0.0:
    print(nonzero_index,value)
  nonzero_index += 1


آزادی
11319 tensor(1., dtype=torch.float64)
مهسا
17139 tensor(1., dtype=torch.float64)


Each vector in the one hot encoding is orthogonal to each other. So the cosine similarity as well as distance between any two vectors are same.
so the nearest words found are the same for the two words. The cosine similarity of each pair of words is 0.

##### Describe advantages and disadvantages of one-hot encoding

#Advantage:
* Simple and Intuitive: One-hot encoding is straightforward to understand and implement. It assigns a unique binary value (1 or 0) to each category, making it easy to interpret.

* Preservation of Information: Each category is represented by its own dimension in the encoded vector, ensuring that no information is lost during the encoding process.

* Compatibility with Machine Learning Algorithms: Many machine learning algorithms, especially those based on linear algebra (e.g., logistic regression, support vector machines), require numerical input data. One-hot encoding provides a way to represent categorical data in a numerical format that can be easily fed into these algorithms.

* Insensitivity to Magnitude: One-hot encoding does not introduce any ordinal relationship between categories. Each category is treated as equally different from others, which can be beneficial in situations where there is no inherent order among categories.
* Robustness to Out-of-Vocabulary (OOV) Words: One-hot encoding inherently handles out-of-vocabulary words or categories by assigning them a dedicated dimension with all zeros. This property makes it robust to unseen categories during training, as the model can still process them during inference without the need for retraining.

#Disadvantages:

* Sparsity: The encoded vectors are sparse, meaning that most elements are zeros. In datasets with many categories, most of the encoded vectors will contain mostly zeros, leading to inefficient memory usage (as I could not implement it by myself because of huge ram usage) and potentially slower computation.


* Lack of Similarity Information: One-hot encoding treats all categories as completely distinct and unrelated, even if some categories may have similarities. This can be problematic in tasks where understanding the relationships between categories is important, such as semantic analysis or recommendation systems.
like this example: the k nearest words where same for all words.

* Curse of Dimensionality: High-dimensional feature spaces can suffer from the curse of dimensionality, where the amount of data required to effectively cover the space grows exponentially with the number of dimensions. This can lead to overfitting and reduced generalization performance of machine learning models.

* Increased Model Complexity: In some cases, the high dimensionality introduced by one-hot encoding can lead to increased model complexity. Models trained on high-dimensional data may require more parameters to learn, leading to longer training times and potentially higher risk of overfitting, especially if the dataset is small. This can make the model less interpretable and harder to understand.


# 3. TF-IDF

In [ ]:
# 1. Find the TF-IDF of all tweets
tf_all = []
idf = {}
# split to words and set idf to zero
for snt in tweets:
    for word in snt.split(' '):
        idf[word] = 0
# Loop through each tweet to calculate TF and update IDF based on reputations
for snt in tweets:
    tf = {}
    for word in snt.split(' '):
        if word not in tf:
            tf[word] = 0
        tf[word] += 1
        idf[word] += 1
    n = len(snt.split(' '))

    # Normalize TF values
    for word in tf:
        tf[word] = float(tf[word]) / n
    tf_all.append(tf)

unique = len(idf)
list_word = {}
for i, word in enumerate(idf):
    # Calculate IDF values
    idf[word] = math.log(unique / idf[word])
    list_word[word] = i

array = np.zeros((len(tweets), unique))
for i, snt in enumerate(tweets):
    for word in snt.split(' '):
        array[i, list_word[word]] = tf_all[i][word] * idf[word]

dict_tf_idf = {}
for i, snt in enumerate(tweets):
    dict_tf_idf[snt] = torch.tensor(array[i])


# 2. Choose one tweet randomly
random_tweet_index = np.random.randint(len(tweets))
random_tweet = tweets[random_tweet_index]
print("Chosen Tweet:", random_tweet)


# 3. Find 10 nearest tweets from the chosen tweet
neighbors = find_k_nearest_neighbors_tensor(random_tweet, dict_tf_idf, 10)
print("Nearest Tweets:", neighbors)

Chosen Tweet: بخاطر حنانه کیا دختر شهید نوشهری
Nearest Tweets: ['برای حنانه کیا', 'برای حنانه', 'برای مهسا برای حنانه برای دختر آینده م', 'بخاطر نیلوفر بخاطر حنانه بخاطر بردیا بخاطر هرخونی که ریختن و نفهمیدیم. برای ایران آزاد.', 'دختر ایران', 'بخاطر اون دختر ده ساله', 'بخاطر ایران', 'امثال سلیمانی ها شهید نیستند مهسا در راه آزادی شهید شد', 'با کیا شدیم ۸۰ میلیون نفر', 'شهید شهید نکنید . اینم ماموراشون که عربن']


##### Describe advantages and disadvantages of TF-IDF

#Advatages:
* Reflects word importance: TF-IDF reflects the importance of a term within a document relative to the entire corpus. This makes it effective in identifying key terms that differentiate one document from another.

* Handles common terms well: TF-IDF effectively penalizes common terms by reducing their weight. Common words like "the," "is," etc., usually appear in many documents and thus have low TF-IDF scores, allowing more focus on meaningful terms.

* Simple and efficient: The calculation of TF-IDF is straightforward and computationally efficient, making it suitable for large datasets and real-time applications.

* Language independent: TF-IDF is language-independent and can be applied to any language. It relies on word frequencies rather than linguistic features, making it versatile across different languages.

* Flexible: TF-IDF allows for flexibility in tuning parameters. For instance, you can adjust the weighting scheme, apply normalization, or set thresholds to suit specific needs or domain requirements.


#Disadvantages:

* Ignores word semantics: TF-IDF treats words as independent units and doesn't consider their semantic relationships. So words with similar meanings but different forms (e.g., "car" and "automobile") are treated as distinct terms, which leads to information loss.

* Sparse Matrix: In large document collections, the matrix representation of TF-IDF can be very sparse, which may increase memory and computational requirements for storage and processing.

* Sensitivity to document length: Longer documents may have higher overall term frequencies, potentially skewing the importance of terms. Techniques like normalization can mitigate this issue to some extent, but it remains a concern in certain contexts.

* Not suitable for capturing context: TF-IDF does not capture the context of words within documents. It treats each term independently, which may not be ideal for tasks where context is crucial, such as natural language understanding or sentiment analysis.

* Requires a representative corpus: TF-IDF relies on a representative corpus to compute document frequencies accurately. In domains where building such a corpus is challenging or where the corpus is not diverse enough, TF-IDF's effectiveness may be compromised.

# 4. Word2Vec

In [ ]:
# 1. train a word2vec model base on all tweets

# Tokenize tweets into words
tokenized_tweets = [tweet.split() for tweet in tweets]
# Use a word2vec model
word2vec_model = Word2Vec(tokenized_tweets, vector_size=100, window=5, min_count=1, workers=4)

# 2. find 10 nearest words from "آزادی"

word="آزادی"
nearest_words = word2vec_model.wv.most_similar(word , topn=10)

# Print the nearest words
for word, similarity in nearest_words:
    print(word, ":", similarity)



##### Describe advantages and disadvantages of Word2Vec


#Advantages:

* Semantic similarity: Word2Vec captures semantic similarities between words effectively. Words with similar meanings are represented by vectors that are close together in the embedding space, enabling better understanding of relationships between words.

* Dimensionality reduction: Word2Vec reduces the dimensionality of word representations compared to traditional one-hot encoding. This dense representation allows for more efficient storage and computation.

* Generalization: Word2Vec embeddings are trained on large corpora, allowing them to capture general semantic relationships across different domains and languages. Pre-trained Word2Vec models can be transferred and fine-tuned for various downstream tasks.

* Speed and scalability: Training Word2Vec models can be computationally efficient, especially compared to more complex neural network architectures. With efficient algorithms like skip-gram and continuous bag-of-words (CBOW), Word2Vec can be trained on large datasets relatively quickly.

* Captures context: Word2Vec considers the context of words within a given window size, allowing it to capture syntactic and semantic relationships based on their co-occurrence patterns in text data.

#Disadvantages:

* Requires large corpus: Word2Vec models require a large corpus of text data to learn meaningful word embeddings. Training on smaller datasets may result in less accurate representations, especially for rare words or domain-specific terms.

* Context window limitation: Word2Vec uses a fixed context window size to capture word context, which may not always capture long-range dependencies or nuanced relationships between words.

* Out-of-vocabulary words: Handling out-of-vocabulary words requires additional techniques such as subword embeddings or handling unknown tokens explicitly.

* Limited to single word semantics: Word2Vec typically represents each word as a single vector, which may not capture multi-word expressions or phrases' semantics effectively. This limitation can affect tasks where understanding the meaning of phrases is crucial.

* Difficulty with polysemy: Word2Vec may struggle to differentiate between multiple meanings of polysemous words since it represents each word with a single vector. As a result, the embeddings may not capture all senses of a word accurately.


# 5. Contextualized embedding

In [ ]:
!pip install transformers[sentencepiece]

In [4]:
# Load model and tokenizer

from transformers import BertModel, BertTokenizer

model_name = "HooshvareLab/bert-base-parsbert-uncased"


In [16]:
from torch.utils.data import DataLoader, Dataset
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
# Read the CSV file with the sentiment data
data = pd.read_csv(SOURCE_DIR)
texts = data['Text']
labels = data['Sentiment']

label_to_number_mapping = {label: number for number, label in enumerate(set(labels))}
print(label_to_number_mapping)
labels = labels.map(label_to_number_mapping)

{'mixed': 0, 'no sentiment expressed': 1, 'positive': 2, 'very negative': 3, 'negative': 4, 'very positive': 5}


resources:
https://www.geeksforgeeks.org/one-hot-encoding-in-nlp/


In [17]:
# Load the BERT tokenizer
from transformers import BertTokenizer
from torch.utils.data import Dataset

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a custom dataset for sentiment classification
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        tweet = self.df.loc[idx, 'Text']
        label = self.df.loc[idx, 'Sentiment']

        # Convert label to number using label_to_number_mapping
        label = label_to_number_mapping[label]

        # Tokenize tweet
        encoding = self.tokenizer.encode_plus(
            tweet,
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors='pt'
        )

        # Flatten input_ids and attention_mask tensors
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:

# Define the BERT model for sentiment classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Define the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
dataset = CustomDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=16)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [21]:
from tqdm import tqdm
import torch.nn as nn

model.to(device)
model.train()

for batch in tqdm(dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}

    outputs = model(**batch)

    loss_function = nn.CrossEntropyLoss()
    loss = loss_function(outputs.logits, batch['labels'])

    print(f"Training Loss: {loss}")

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

print("Training complete!")

  0%|          | 0/1250 [00:00<?, ?it/s]

Training Loss: 1.7344597578048706


  0%|          | 2/1250 [00:01<16:03,  1.29it/s]

Training Loss: 1.8218648433685303


  0%|          | 3/1250 [00:02<11:36,  1.79it/s]

...

Training Loss: 1.0598869323730469


100%|█████████▉| 1249/1250 [07:01<00:00,  2.94it/s]

Training Loss: 1.1596072912216187


100%|██████████| 1250/1250 [07:01<00:00,  2.96it/s]

Training complete!


In [28]:
# 2. find 10 nearest words from "آزادی"

word = 'آزادی'
# Get the embedding of the word
with torch.no_grad():
    word_embedding = model(tokenizer.encode(word, return_tensors='pt').to(device))[0][0, :].cpu().numpy()

similar_words = []
# Calculate the cosine similarity
# checking all other words
for word in tqdm(tokenizer.get_vocab()):
    with torch.no_grad():
        word_embedding = model(tokenizer.encode(word, return_tensors='pt').to(device))[0][0, :].cpu().numpy()
    similar_word = 1 - cosine_similarity(word_embedding, word_embedding)
    similar_words.append((word, similar_word))

# Sort the found words by similarity
sorted_similar = sorted(similar_words, key=lambda x: x[1], reverse=True)

print(sorted_similar)
# Get the top 10 similar words
top_10_similars = sorted_similar[:10]

print(top_10_similars)

100%|██████████| 30522/30522 [05:51<00:00, 86.86it/s]

[('[unused266]', 2.384185791015625e-07), ('[unused549]', 2.384185791015625e-07), ('[unused573]', 2.384185791015625e-07), (';', 2.384185791015625e-07), ('ན', 2.384185791015625e-07), ('》', 2.384185791015625e-07), ('古', 2.384185791015625e-07), ('田', 2.384185791015625e-07), ('白', 2.384185791015625e-07), ('禾', 2.384185791015625e-07), ('first', 2.384185791015625e-07), ('people', 2.384185791015625e-07), ('de', 2.384185791015625e-07), ('2006', 2.384185791015625e-07), ('##b', 2.384185791015625e-07), ('anything', 2.384185791015625e-07), ('moment', 2.384185791015625e-07), ('available', 2.384185791015625e-07), ('1986', 2.384185791015625e-07), ('question', 2.384185791015625e-07), ('greater', 2.384185791015625e-07), ('difficult', 2.384185791015625e-07), ('technical', 2.384185791015625e-07), ('participated', 2.384185791015625e-07), ('armed', 2.384185791015625e-07), ('opportunity', 2.384185791015625e-07), ('singing', 2.384185791015625e-07), ('temperature', 2.384185791015625e-07), ('turkey', 2.38418579

##### Describe advantages and disadvantages of Contextualized embedding

#Advantages:

BERT works well for task-specific models. The state of the art model, BERT, has been trained on a large corpus, making it easier for smaller, more defined nlp tasks.

Metrics can be fine-tuned and be used immediately.

The accuracy of the model is outstanding because it is frequently updated. You can achieve this with successful fine-tuning training.

The BERT model is available and pre-trained in more than 100 languages. This can be useful for projects that are not English-based.

#Disadvantages:
The main drawbacks of using BERT and other big neural language models is the computational resources needed to train/fine-tune and make inferences.

Most of the drawbacks of BERT can be linked to its size. While training the data on a large corpus significantly helps how the computer predicts and learns, there is also another side to it. They include:

The model is large because of the training structure and corpus.

It is slow to train because it is big and there are a lot of weights to update.

It is expensive. It requires more computation because of its size, which comes at a cost.

Disadvantages:
